In [101]:
import csv
import re
import os
import pandas as pd
import numpy as np
import xlsxwriter

from datetime import datetime

In [102]:
file_root = './files'
church_export_root = './church'
event_export_root = './events'

event_exclusion_data_cols = [
    'Submission Date',
    'Form Submission Date',
    'I am registering as a',
    'Are you Participating or Spectating?',
    'District',
    'Birthday',
    'Street Address',
    'Street Address Line 2',
    'City',
    'State / Province',
    'Postal / Zip Code',
    'Cell Phone',
    'Your Email',
    'Gender',
    'Shirt Size',
    'First Name.1',
    'Last Name.1',
    'What is your youth leader/pastor\'s email?',
    'First Name.2',
    'Last Name.2',
    'Parent/Guardian Email',
    'Parent Cell Phone',
    'Please list any medical problems',
    'Please list any allergies',
    'Please list past surgeries',
    'Please list any medications and dosage you will be taking',
    'First Name.3',
    'Last Name.3',
    'Home Phone',
    'Work Phone',
    'Contact Phone',
    'Insurance Company',
    'Policy Number #',
    'Do you plan on attending TNT@TNU April 13th-April 16th?',
    'Payment'
]

write_file_exlusion = ['First Name', 'Last Name', 'Grade Level', 'Age Level Individual Sport', 'Age Level', 'What church are you a part of?']

event_categories = [
    'Art',
    'Creative Ministries',
    'Creative Writing',
    'Speech',
    'Academics',
    'Vocal Music',
    'Instrumental Music',
    'Individual Sports',
    'Team Sports',
    'Quizzing'
]

church_list = [
    "Ashland City Faith Community",
    "Bell Road",
    "Bent Knee Cowboy",
    "Body of Christ",
    "Bread of Life Community",
    "Bridge Point",
    "Brookhaven First",
    "Clarksdale",
    "Clarksville First Community of Hope",
    "Clarksville Grace",
    "Clarksville Hope Riders Biker Church",
    "Cleveland First",
    "Columbia First",
    "Columbia Grace",
    "Columbus First",
    "Community Fellowship",
    "Concord Community",
    "Covington",
    "Crossroads",
    "Crossroads Community",
    "Crystal Springs",
    "DaySpring Community",
    "Dickson Lighthouse",
    "Donelson",
    "Dover",
    "Elmore",
    "Erin",
    "Fellowship",
    "Fly",
    "Foundry",
    "Franklin Community Faith",
    "Fulton First",
    "Gallatin First",
    "Gateway Community",
    "Glencliff Hispanic",
    "Gloster",
    "Goodlettsville",
    "Gray's Chapel",
    "Grenada First",
    "Griffin's Chapel",
    "Gulf Coast Family",
    "Harmony",
    "Hattiesburg First",
    "Hazkehurst",
    "Hendersonville",
    "Hermitage",
    "Highland",
    "Hillside Community",
    "Igelsia Del Nazareno Eben-Ezer",
    "Igelsia Hispana Betel de Gallatin",
    "Jason Chapel",
    "Joelton",
    "Jones Chapel",
    "Kingston Springs",
    "Laurel Bresee",
    "Laurel First",
    "Lewisburg First",
    "Libertad en Cristo",
    "Lifepointe Community",
    "Long Creek",
    "Madison",
    "Madison Hispanic",
    "Magnolia",
    "Memphis Calvary",
    "Memphis Eastside",
    "Memphis Emmanuel",
    "Memphis Friendship",
    "Memphis Grace",
    "Memphis Holiness Tabernacle",
    "Memphis Hope Tabernacle",
    "Memphis Refreshing Springs",
    "Meridian Fitkins Memorial",
    "Meridian Northside",
    "Millington",
    "Mount Wesley-Akin Chapel",
    "Nashville Blakemore",
    "Nashville Christ",
    "Nashville First (Main Campus)",
    "Nashville First (Bethel Campus)",
    "Nashville First (Calvary) Korean",
    "Nashville Grace",
    "Nashville Inglewood",
    "Nashville Kaleo",
    "Nashville Trinity",
    "New Albany First",
    "New Vision",
    "Old Hickory",
    "Oxford",
    "Palabra de Vida",
    "Paris",
    "Park Lane",
    "Pearson Chapel",
    "Pine Hill",
    "Pleasant Point",
    "Portland",
    "Prentiss",
    "Quitman",
    "Ripley Hispanic",
    "Rosebloom",
    "Savannah",
    "Sentobia N.O. Nabors Memorial",
    "Spring Hill Restoration Community",
    "Springfield First",
    "Starkville Wesley Community",
    "Trevecca Community",
    "Union",
    "Vicksburg First/Vicksburg Hispanic",
    "Victory Praise and Worship Center",
    "Waverly",
    "Way of the Cross",
    "Waynesboro Trinity",
    "Westwind Community of Faith",
    "Woodlawn Community"
]

In [103]:

files = os.listdir(file_root)
files = [f for f in files if os.path.isfile(f'{file_root}/{f}')] 
files

['Momentum_Registration_-_20222022-11-03_22_06_12.csv',
 'Momentum_Registration.csv']

In [104]:
def write_csv(df, df_name, filename):
    if df_name not in write_file_exlusion:
        df.to_csv(f'{filename}.csv')

In [105]:
def write_excel(df, df_name, filename, indexed=False):
    if df_name not in write_file_exlusion:
        df.to_excel(f'{filename}.xlsx', index=indexed, engine='xlsxwriter')

In [106]:
def strip_filename_from_download_date(filename):
    filename_parts = filename.split('_-_')
    return filename_parts[0]

strip_filename_from_download_date(files[0])

'Momentum_Registration'

In [155]:
from datetime import date


def create_student_categories(file, filename):
   df_original = pd.read_csv(f'{file_root}/{file}')

   cols = df_original.columns;
    
   category_cols = [];
   for col in cols:
      if col not in event_exclusion_data_cols:
         category_cols.append(col)
    
   for category_col in category_cols:
      loc_cols = ['First Name', 'Last Name', 'What church are you a part of?']
      group_by_cols = [category_col]
      if 'sport' in category_col.lower():
         loc_cols.append('Age Level Individual Sport')
      else:
         loc_cols.append('Age Level')

      if category_col == 'Individual Sports':
         group_by_cols.append('Age Level Individual Sport')
      if category_col == 'Team Sports':
         group_by_cols.append('What church are you a part of?')
      if 'Music' in category_col:
         group_by_cols.append('What church are you a part of?')

      df = df_original.copy(deep=True)
      df = df.loc[:, [*loc_cols, category_col]].dropna(subset=[category_col])

      if category_col in event_categories:
         df[category_col] = df.loc[:, category_col].apply(lambda x: x.split('\n'))
         df_dedeup = df.drop_duplicates(subset=loc_cols)
         df_indexed = df_dedeup.explode(category_col).groupby([*group_by_cols, 'First Name', 'Last Name', 'What church are you a part of?']).sum()
         display(df_indexed)
         
         export_path = f"{event_export_root}/{date.today()}_{category_col.replace(' ', '_').lower()}"
         write_excel(df_indexed, category_col, export_path, indexed=True)

create_student_categories(files[0],strip_filename_from_download_date(files[0]))


Age Level
Art     First Name          Last Name     What church are you a part of?                                    
3-D Art Alejandra Elizabeth Lopez Catalan Glencliff Hispanic                 Early Youth (Grade 9 and below)
        Alison              Amos          Nashville First (Main Campus)      Early Youth (Grade 9 and below)
        Allyson             Thomas        Clarksville Grace                   Senior Youth (Grade 10 and up)
        Bethany             Thomas        Clarksville Grace                   Senior Youth (Grade 10 and up)
        David               Wood          Nashville Grace                     Senior Youth (Grade 10 and up)
...                                                                                                      ...
Water   David               Wood          Nashville Grace                     Senior Youth (Grade 10 and up)
        Gabriella           Miller        Gallatin First                     Early Youth (Grade 9 and below)
                            Phillips      Spring Hill Restoration Community   Senior Youth (Grade 10 and up)
        Ingrid              Balderrama    Igelsia Del Nazareno Eben-Ezer      Senior Youth (Grade 10 and up)
        Lincoln             Burch         Hermitage                           Senior Youth (Grade 10 and up)

[168 rows x 1 columns]

Age Level
Creative Ministries First Name Last Name What church are you a part of?                                    
Dramatic Monologue  Addie      Bennett   Nashville First (Main Campus)      Early Youth (Grade 9 and below)
                    Anna       Gernand   Gateway Community                   Senior Youth (Grade 10 and up)
                    Gabriella  Phillips  Spring Hill Restoration Community   Senior Youth (Grade 10 and up)
                    Wes        Twining   Nashville First (Main Campus)       Senior Youth (Grade 10 and up)
                    klarissa   mcfarland Fly                                Early Youth (Grade 9 and below)
...                                                                                                     ...
Video Production    Madison    Willis    Memphis Friendship                 Early Youth (Grade 9 and below)
                    Malaya     Fields    Memphis Friendship                 Early Youth (Grade 9 and below)
                    Taylor     Love      Memphis Friendship                 Early Youth (Grade 9 and below)
                    Trinity    Love      Memphis Friendship                 Early Youth (Grade 9 and below)
                    Xzavion    Walton    Memphis Friendship                  Senior Youth (Grade 10 and up)

[77 rows x 1 columns]

Age Level
Creative Writing First Name      Last Name What church are you a part of?                                    
Poetry           Addie           Bennett   Nashville First (Main Campus)      Early Youth (Grade 9 and below)
                 Alice           Gooding   Clarksville Grace                  Early Youth (Grade 9 and below)
                 Allyson         Thomas    Clarksville Grace                   Senior Youth (Grade 10 and up)
                 Anna            Zirschky  Spring Hill Restoration Community  Early Youth (Grade 9 and below)
                 Ashley          Harris    Hendersonville                      Senior Youth (Grade 10 and up)
                 Beatrice        Spun      Erin                               Early Youth (Grade 9 and below)
                 Caroline        Foster    Nashville First (Main Campus)       Senior Youth (Grade 10 and up)
                 Drew            Johnson   Nashville First (Main Campus)       Senior Youth (Grade 10 and up)
                 Edwin           Reyes     Igelsia Del Nazareno Eben-Ezer     Early Youth (Grade 9 and below)
                 Ian             Gibson    Columbia Grace                     Early Youth (Grade 9 and below)
                 Jason           Burch     Hermitage                           Senior Youth (Grade 10 and up)
                 Lauren          Cline     Hattiesburg First                  Early Youth (Grade 9 and below)
                 Maria Guadalupe Sis Lucas Glencliff Hispanic                 Early Youth (Grade 9 and below)
                 Melissa         De Paz    Glencliff Hispanic                 Early Youth (Grade 9 and below)
                 Nataly          Rodriguez Old Hickory                        Early Youth (Grade 9 and below)
                 klarissa        mcfarland Fly                                Early Youth (Grade 9 and below)
                 olivia          Nichols   Nashville Grace                     Senior Youth (Grade 10 and up)
Prose            Addie           Bennett   Nashville First (Main Campus)      Early Youth (Grade 9 and below)
                 Aidan           Barnes    Concord Community                   Senior Youth (Grade 10 and up)
                 Beatrice        Spun      Erin                               Early Youth (Grade 9 and below)
                 Ella            Lawrence  Goodlettsville                      Senior Youth (Grade 10 and up)
                 Julianne        Erlewine  Gallatin First                     Early Youth (Grade 9 and below)
                 Kaylee          Rumbley   Nashville Grace                     Senior Youth (Grade 10 and up)
                 olivia          Nichols   Nashville Grace                     Senior Youth (Grade 10 and up)

Age Level
Speech              First Name Last Name  What church are you a part of?                                      
Impromptu           Addie      Bennett    Nashville First (Main Campus)        Early Youth (Grade 9 and below)
                    Isabelle   Rector     Gallatin First                        Senior Youth (Grade 10 and up)
Oral Interpretation Addie      Bennett    Nashville First (Main Campus)        Early Youth (Grade 9 and below)
                    Addyson    Underwood  Concord Community                    Early Youth (Grade 9 and below)
                    Anna Grace Rector     Gallatin First                       Early Youth (Grade 9 and below)
                    Avery      Underwood  Concord Community                    Early Youth (Grade 9 and below)
                    Charley    Lawrence   Goodlettsville                       Early Youth (Grade 9 and below)
                    Emory      Duranceau  Clarksville First Community of Hope  Early Youth (Grade 9 and below)
                    Ian        Gibson     Columbia Grace                       Early Youth (Grade 9 and below)
                    Isabelle   Rector     Gallatin First                        Senior Youth (Grade 10 and up)
Original Oratory    Addie      Bennett    Nashville First (Main Campus)        Early Youth (Grade 9 and below)
                    Edwin      Reyes      Igelsia Del Nazareno Eben-Ezer       Early Youth (Grade 9 and below)
                    Isabelle   Rector     Gallatin First                        Senior Youth (Grade 10 and up)
Storytelling        Alice      Gooding    Clarksville Grace                    Early Youth (Grade 9 and below)
                    Isabelle   Rector     Gallatin First                        Senior Youth (Grade 10 and up)
                    Julianne   Erlewine   Gallatin First                       Early Youth (Grade 9 and below)
                    Keyira     Jackson    Hattiesburg First                    Early Youth (Grade 9 and below)
                    Lorelai    Cochran    Waverly                              Early Youth (Grade 9 and below)
                    Taylor     Banes      Hattiesburg First                    Early Youth (Grade 9 and below)
                    klarissa   mcfarland  Fly                                  Early Youth (Grade 9 and below)

Age Level
Academics                First Name          Last Name       What church are you a part of?                                      
Math                     Adeline             Miller          Goodlettsville                       Early Youth (Grade 9 and below)
                         Aidan               Barnes          Concord Community                     Senior Youth (Grade 10 and up)
                         Alejandra Elizabeth Lopez Catalan   Glencliff Hispanic                   Early Youth (Grade 9 and below)
                         Alice               Gooding         Clarksville Grace                    Early Youth (Grade 9 and below)
                         Allister            Griffin         Spring Hill Restoration Community     Senior Youth (Grade 10 and up)
                         Anna                Zirschky        Spring Hill Restoration Community    Early Youth (Grade 9 and below)
                         Benjamin            Wentworth       Trevecca Community                   Early Youth (Grade 9 and below)
                         Bennett             Hunter          Spring Hill Restoration Community    Early Youth (Grade 9 and below)
                         Edwin               Reyes           Igelsia Del Nazareno Eben-Ezer       Early Youth (Grade 9 and below)
                         Eliseo              Herrera Xitumul Glencliff Hispanic                    Senior Youth (Grade 10 and up)
                         Emily               Storie          Nashville First (Main Campus)        Early Youth (Grade 9 and below)
                         Emory               Duranceau       Clarksville First Community of Hope  Early Youth (Grade 9 and below)
                         Eva                 Webb            Gateway Community                     Senior Youth (Grade 10 and up)
                         Gabriella           Phillips        Spring Hill Restoration Community     Senior Youth (Grade 10 and up)
                         Hayden              Amos            Nashville First (Main Campus)        Early Youth (Grade 9 and below)
                         Ian                 Gibson          Columbia Grace                       Early Youth (Grade 9 and below)
                         Isabel              Lopez-Muñoz     Gallatin First                        Senior Youth (Grade 10 and up)
                         Isabelle            Rector          Gallatin First                        Senior Youth (Grade 10 and up)
                         Jaiden              Cochran         Waverly                               Senior Youth (Grade 10 and up)
                         Jason               Burch           Hermitage                             Senior Youth (Grade 10 and up)
                         Jenna               Hartsock        Clarksville Grace                    Early Youth (Grade 9 and below)
                         Jose                Camaja Perez    Glencliff Hispanic                   Early Youth (Grade 9 and below)
                         Joshten             Pascacio        Igelsia Del Nazareno Eben-Ezer        Senior Youth (Grade 10 and up)
                         Julianne            Erlewine        Gallatin First                       Early Youth (Grade 9 and below)
                         Kathleen            Jeronimo        Gallatin First                       Early Youth (Grade 9 and below)
                         Kinley              Hodge           Gallatin First                       Early Youth (Grade 9 and below)
                         Maggie              Shelton         Concord Community                    Early Youth (Grade 9 and below)
                         Maria Guadalupe     Sis Lucas       Glencliff Hispanic                   Early Youth (Grade 9 and below)
                         Mason               Roberson        Spring Hill Restoration Community     Senior Youth (Grade 10 and up)
                         Melissa             De Paz          Glencliff Hispanic           

Age Level
Vocal Music        What church are you a part of? First Name Last Name What church are you a part of?                                 
Singer/Song Writer Clarksville Grace              Alice      Gooding   Clarksville Grace               Early Youth (Grade 9 and below)
                                                  Allyson    Thomas    Clarksville Grace                Senior Youth (Grade 10 and up)
                                                  Bethany    Thomas    Clarksville Grace                Senior Youth (Grade 10 and up)
                   Concord Community              Kyla       Watson    Concord Community               Early Youth (Grade 9 and below)
                   Gallatin First                 Julianne   Erlewine  Gallatin First                  Early Youth (Grade 9 and below)
...                                                                                                                                ...
Worship Band       Memphis Grace                  Veronica   Ndikumana Memphis Grace                   Early Youth (Grade 9 and below)
                   Nashville First (Main Campus)  Chloe      Carawan   Nashville First (Main Campus)   Early Youth (Grade 9 and below)
                   Nashville Grace                David      Wood      Nashville Grace                  Senior Youth (Grade 10 and up)
                                                  Natalie    Reed      Nashville Grace                 Early Youth (Grade 9 and below)
                                                  Stephen    Purtee    Nashville Grace                  Senior Youth (Grade 10 and up)

[170 rows x 1 columns]

Age Level
Instrumental Music           What church are you a part of?      First Name Last Name    What church are you a part of?                                      
Instrumental Brass Solo      Crossroads                          Amie       Daniel       Crossroads                           Early Youth (Grade 9 and below)
                             Erin                                Jacob      Garza        Erin                                  Senior Youth (Grade 10 and up)
                                                                 kloe       shelton      Erin                                 Early Youth (Grade 9 and below)
                             Gateway Community                   Cooper     Landry       Gateway Community                    Early Youth (Grade 9 and below)
                                                                 Eva        Webb         Gateway Community                     Senior Youth (Grade 10 and up)
                             Spring Hill Restoration Community   Bennett    Hunter       Spring Hill Restoration Community    Early Youth (Grade 9 and below)
                             Trevecca Community                  Alex       Yusi         Trevecca Community                   Early Youth (Grade 9 and below)
Instrumental Group           Clarksville Grace                   Jenna      Hartsock     Clarksville Grace                    Early Youth (Grade 9 and below)
                             Columbia Grace                      Ian        Gibson       Columbia Grace                       Early Youth (Grade 9 and below)
Instrumental Percussion Solo Clarksville Grace                   Bethany    Thomas       Clarksville Grace                     Senior Youth (Grade 10 and up)
                             Columbia Grace                      Jackson    Johns        Columbia Grace                       Early Youth (Grade 9 and below)
                             Erin                                Benjamin   Meadow       Erin                                  Senior Youth (Grade 10 and up)
                             Gallatin First                      Wyatt      Johnson      Gallatin First                       Early Youth (Grade 9 and below)
                             Goodlettsville                      Matthew    Kropf        Goodlettsville                       Early Youth (Grade 9 and below)
                             Hattiesburg First                   David      Gavel        Hattiesburg First                    Early Youth (Grade 9 and below)
                             Nashville First (Main Campus)       Chloe      Carawan      Nashville First (Main Campus)        Early Youth (Grade 9 and below)
Instrumental Strings Solo    Erin                                Elijah     Spun         Erin                                 Early Youth (Grade 9 and below)
                                                                 Owen       Ziegler      Erin                                  Senior Youth (Grade 10 and up)
                             Gateway Community                   Elijah     McMillian    Gateway Community                     Senior Youth (Grade 10 and up)
                             Hendersonville                      Kenny      Gerdes       Hendersonville                        Senior Youth (Grade 10 and up)
                             Nashville Grace                     Keegan     Welch        Nashville Grace                       Senior Youth (Grade 10 and up)
                                                                 Keegan     Welch        Nashville Grace                       Senior Youth (Grade 10 and up)
                             Spring Hill Restoration Community   Allister   Griffin      Spring Hill Restoration Community     Senior Youth (Grade 10 and up)
Keyboard Solo                Clarksville Grace                   Allyson    Thomas       Clarksville Grace                     Senior Youth (Grade 10 and up)
                             Erin       

Empty DataFrame
Columns: []
Index: [(* Table Tennis, Junior High (ages 11-13), Adeline, Miller, Goodlettsville), (* Table Tennis, Junior High (ages 11-13), Antonio, Paulson, Goodlettsville), (* Table Tennis, Junior High (ages 11-13), Aubrey, Baggett, Columbia Grace), (* Table Tennis, Junior High (ages 11-13), Ben , Meikrantz, Hendersonville), (* Table Tennis, Junior High (ages 11-13), Eli, Ward, Memphis Grace), (* Table Tennis, Junior High (ages 11-13), Emry, Thompson, Gateway Community), (* Table Tennis, Junior High (ages 11-13), Jackson, Johns, Columbia Grace), (* Table Tennis, Junior High (ages 11-13), Jonah, Britton, Memphis Grace), (* Table Tennis, Junior High (ages 11-13), Lola Jane, Galloway, Nashville First (Main Campus)), (* Table Tennis, Middle High (ages 14-15), Avery, Underwood , Concord Community), (* Table Tennis, Middle High (ages 14-15), Claude, Galloway, Nashville First (Main Campus)), (* Table Tennis, Middle High (ages 14-15), Elijah, McMillian, Gateway Community), (* Table Tennis, Middle High (ages 14-15), Grey, Gernand, Gateway Community), (* Table Tennis, Middle High (ages 14-15), John, Wilkes, Nashville Grace), (* Table Tennis, Middle High (ages 14-15), Jonathan, Griffin, Nashville First (Main Campus)), (* Table Tennis, Middle High (ages 14-15), Landen, Boyd, Gateway Community), (* Table Tennis, Middle High (ages 14-15), Lily, Quick, Nashville First (Main Campus)), (* Table Tennis, Middle High (ages 14-15), Nataly, Rodriguez, Old Hickory), (* Table Tennis, Middle High (ages 14-15), William, Voss, Fly), (* Table Tennis, Senior High (ages 16-19), Addie , Peeples, Crossroads), (* Table Tennis, Senior High (ages 16-19), Allister , Griffin, Spring Hill Restoration Community), (* Table Tennis, Senior High (ages 16-19), Charles, Galloway, Nashville First (Main Campus)), (* Table Tennis, Senior High (ages 16-19), David, Wood, Nashville Grace), (* Table Tennis, Senior High (ages 16-19), Jason, Burch, Hermitage), (* Table Tennis, Senior High (ages 16-19), Kaden, Boyd, Gateway Community), (* Table Tennis, Senior High (ages 16-19), Kaylee, Rumbley, Nashville Grace), (* Table Tennis, Senior High (ages 16-19), Sam, Ruscio, Crossroads), (* Table Tennis, Senior High (ages 16-19), Warren, Bailey, Nashville First (Main Campus)), (* Table Tennis, Senior High (ages 16-19), Wes, Twining, Nashville First (Main Campus)), (* Tennis, Junior High (ages 11-13), Lucy , Howell, Nashville First (Main Campus)), (* Tennis, Middle High (ages 14-15), Avery, Underwood , Concord Community), (* Tennis, Middle High (ages 14-15), Dylan, Johnson, Nashville First (Main Campus)), (* Tennis, Senior High (ages 16-19), Cole, Petroff, Hermitage), (* Tennis, Senior High (ages 16-19), Giles, Knop, Hermitage), (* Tennis, Senior High (ages 16-19), Hudson, Stearns, Fly), (* Tennis, Senior High (ages 16-19), Jason, Burch, Hermitage), (* Tennis, Senior High (ages 16-19), faith , gill, Crossroads), (** Chess, Junior High (ages 11-13), Alice, Gooding, Clarksville Grace), (** Chess, Junior High (ages 11-13), Annalise, Thornsberry, Erin), (** Chess, Junior High (ages 11-13), Austin, Reigard, Hermitage), (** Chess, Junior High (ages 11-13), Eli, Ward, Memphis Grace), (** Chess, Junior High (ages 11-13), Hayden, Amos, Nashville First (Main Campus)), (** Chess, Junior High (ages 11-13), Jose , Camaja Perez, Glencliff Hispanic), (** Chess, Middle High (ages 14-15), Anna, Zirschky, Spring Hill Restoration Community), (** Chess, Middle High (ages 14-15), Hunter, Moore, Erin), (** Chess, Middle High (ages 14-15), Landen, Boyd, Gateway Community), (** Chess, Senior High (ages 16-19), Carson, Conwell, Erin), (** Chess, Senior High (ages 16-19), Eliseo, Herrera Xitumul, Glencliff Hispanic), (** Chess, Senior High (ages 16-19), Hunter , Jenkins , Trevecca Community), (** Chess, Senior High (ages 16-19), Isabelle, Rector, Gallatin First), (** Chess, Senior High (ages 16-19), Jason, Burch, Hermitage), (** Chess, Senior High (ages 16-19), Kaden, Boyd, Gateway Community), (**

Age Level Individual Sport
Team Sports        What church are you a part of?    First Name Last Name What church are you a part of?                              
* Co-Ed Dodgeball  Clarksville Grace                 Alice      Gooding   Clarksville Grace                   Junior High (ages 11-13)
                                                     Jackson    Smith     Clarksville Grace                   Middle High (ages 14-15)
                                                     Serenity   Halstead  Clarksville Grace                   Senior High (ages 16-19)
                                                     alan       minor     Clarksville Grace                   Junior High (ages 11-13)
                   Columbia Grace                    Ian        Gibson    Columbia Grace                      Middle High (ages 14-15)
...                                                                                                                                ...
** Co-Ed Gaga Ball Spring Hill Restoration Community Allister   Griffin   Spring Hill Restoration Community   Senior High (ages 16-19)
                   Trevecca Community                Aiden      Pierc     Trevecca Community                  Middle High (ages 14-15)
                                                     Cooper     Brown     Trevecca Community                  Junior High (ages 11-13)
                                                     Rhylee     Colonello Trevecca Community                  Junior High (ages 11-13)
                   Waverly                           Ainsley    Forester  Waverly                             Junior High (ages 11-13)

[537 rows x 1 columns]

Age Level
Quizzing              First Name Last Name   What church are you a part of?                                      
* "A" League Quizzing Abbigale   Savage      Crossroads                           Early Youth (Grade 9 and below)
                      Aundrea    Livingston  Erin                                 Early Youth (Grade 9 and below)
                      Ben        Meikrantz   Hendersonville                       Early Youth (Grade 9 and below)
                      Benjamin   Staples     Memphis Calvary                      Early Youth (Grade 9 and below)
                      Carson     Wilkins     Waverly                              Early Youth (Grade 9 and below)
                      Chandler   Pearce      Goodlettsville                        Senior Youth (Grade 10 and up)
                      Chloe      Borja       Fly                                  Early Youth (Grade 9 and below)
                      Connor     Thomas      Nashville First (Main Campus)         Senior Youth (Grade 10 and up)
                      Edwin      Reyes       Igelsia Del Nazareno Eben-Ezer       Early Youth (Grade 9 and below)
                      Hudson     Stearns     Fly                                   Senior Youth (Grade 10 and up)
                      Ian        Gibson      Columbia Grace                       Early Youth (Grade 9 and below)
                      Isbell     Daniel      Hermitage                             Senior Youth (Grade 10 and up)
                      Kaden      Boyd        Gateway Community                     Senior Youth (Grade 10 and up)
                      Landen     Boyd        Gateway Community                     Senior Youth (Grade 10 and up)
                      Stephen    Purtee      Nashville Grace                       Senior Youth (Grade 10 and up)
                      klarissa   mcfarland   Fly                                  Early Youth (Grade 9 and below)
* "B" League Quizzing Alice      Gooding     Clarksville Grace                    Early Youth (Grade 9 and below)
                      Beatrice   Spun        Erin                                 Early Youth (Grade 9 and below)
                      Emory      Duranceau   Clarksville First Community of Hope  Early Youth (Grade 9 and below)
                      Max        Williams    Trevecca Community                    Senior Youth (Grade 10 and up)
                      Nik        Ware        Hermitage                             Senior Youth (Grade 10 and up)
                      Rudy       Colonello   Trevecca Community                   Early Youth (Grade 9 and below)

In [108]:
def create_church_info_sheets(file, filename):
    df_original = pd.read_csv(f'{file_root}/{file}')

    df = df_original.copy(deep=True)
    cols = df_original.columns;
    
    category_cols = [];
    for col in cols:
        if col not in event_exclusion_data_cols:
            category_cols.append(col)
    
    for category_col in category_cols:
        if category_col in event_categories:
            df[category_col] = df.loc[:, category_col].str.split('\n')

    cols = ['First Name', 'Last Name', 'Grade Level', 'What church are you a part of?', *event_categories, 'First Name.1', 'Last Name.1', 'What is your youth leader/pastor\'s email?', 'Payment']
    df = df.loc[:, cols].fillna('')
    df = df.groupby(['What church are you a part of?', 'First Name', 'Last Name'], group_keys=False).apply(lambda x: x)
    df.drop_duplicates(subset=['First Name', 'Last Name', 'Grade Level'], inplace=True)
    df.reset_index(inplace=True)

    for church in church_list:
        df_church = df.loc[df['What church are you a part of?'] == church].copy(deep=True)
        df_church['Payment'] = df_church['Payment'].apply(lambda x: int(re.findall("[0-9]+", x)[0]) if re.findall("[0-9]+", x) else 0)
        df_church.rename(columns = {"First Name.1": "Youth Leader First Name", "Last Name.1": "Youth Leader Last Name"}, inplace = True)
        if not df_church.empty:
            df_church.loc['Total',:] = df_church.sum(axis=0, numeric_only=True)
            export_path = f"{church_export_root}/{date.today()}_{church.replace(' ', '_').lower()}"
            write_excel(df_church, church, export_path)


# create_church_info_sheets(files[0], strip_filename_from_download_date(files[0]))

In [109]:
for file in files:
    filename = strip_filename_from_download_date(file)
    
    create_student_categories(file, filename)
    create_church_info_sheets(file, filename)

Age Level
Art     First Name          Last Name     What church are you a part of?                                    
3-D Art Alejandra Elizabeth Lopez Catalan Glencliff Hispanic                 Early Youth (Grade 9 and below)
        Alison              Amos          Nashville First (Main Campus)      Early Youth (Grade 9 and below)
        Allyson             Thomas        Clarksville Grace                   Senior Youth (Grade 10 and up)
        Bethany             Thomas        Clarksville Grace                   Senior Youth (Grade 10 and up)
        David               Wood          Nashville Grace                     Senior Youth (Grade 10 and up)
...                                                                                                      ...
Water   David               Wood          Nashville Grace                     Senior Youth (Grade 10 and up)
        Gabriella           Miller        Gallatin First                     Early Youth (Grade 9 and below)
                            Phillips      Spring Hill Restoration Community   Senior Youth (Grade 10 and up)
        Ingrid              Balderrama    Igelsia Del Nazareno Eben-Ezer      Senior Youth (Grade 10 and up)
        Lincoln             Burch         Hermitage                           Senior Youth (Grade 10 and up)

[169 rows x 1 columns]

Age Level
Creative Ministries First Name Last Name What church are you a part of?                                    
Dramatic Monologue  Addie      Bennett   Nashville First (Main Campus)      Early Youth (Grade 9 and below)
                    Anna       Gernand   Gateway Community                   Senior Youth (Grade 10 and up)
                    Gabriella  Phillips  Spring Hill Restoration Community   Senior Youth (Grade 10 and up)
                    Wes        Twining   Nashville First (Main Campus)       Senior Youth (Grade 10 and up)
                    klarissa   mcfarland Fly                                Early Youth (Grade 9 and below)
...                                                                                                     ...
Video Production    Madison    Willis    Memphis Friendship                 Early Youth (Grade 9 and below)
                    Malaya     Fields    Memphis Friendship                 Early Youth (Grade 9 and below)
                    Taylor     Love      Memphis Friendship                 Early Youth (Grade 9 and below)
                    Trinity    Love      Memphis Friendship                 Early Youth (Grade 9 and below)
                    Xzavion    Walton    Memphis Friendship                  Senior Youth (Grade 10 and up)

[79 rows x 1 columns]

Age Level
Creative Writing First Name      Last Name What church are you a part of?                                                      
Poetry           Addie           Bennett   Nashville First (Main Campus)                        Early Youth (Grade 9 and below)
                 Alice           Gooding   Clarksville Grace                                    Early Youth (Grade 9 and below)
                 Allyson         Thomas    Clarksville Grace                                     Senior Youth (Grade 10 and up)
                 Anna            Zirschky  Spring Hill Restoration Community                    Early Youth (Grade 9 and below)
                 Ashley          Harris    Hendersonville                                        Senior Youth (Grade 10 and up)
                 Beatrice        Spun      Erin                                                 Early Youth (Grade 9 and below)
                 Caroline        Foster    Nashville First (Main Campus)                         Senior Youth (Grade 10 and up)
                 Drew            Johnson   Nashville First (Main Campus)                         Senior Youth (Grade 10 and up)
                 Edwin           Reyes     Igelsia Del Nazareno Eben-Ezer     Early Youth (Grade 9 and below)Early Youth (Gr...
                 Ian             Gibson    Columbia Grace                                       Early Youth (Grade 9 and below)
                 Jason           Burch     Hermitage                                             Senior Youth (Grade 10 and up)
                 Lauren          Cline     Hattiesburg First                                    Early Youth (Grade 9 and below)
                 Maria Guadalupe Sis Lucas Glencliff Hispanic                                   Early Youth (Grade 9 and below)
                 Melissa         De Paz    Glencliff Hispanic                                   Early Youth (Grade 9 and below)
                 Nataly          Rodriguez Old Hickory                                          Early Youth (Grade 9 and below)
                 klarissa        mcfarland Fly                                                  Early Youth (Grade 9 and below)
                 olivia          Nichols   Nashville Grace                                       Senior Youth (Grade 10 and up)
Prose            Addie           Bennett   Nashville First (Main Campus)                        Early Youth (Grade 9 and below)
                 Aidan           Barnes    Concord Community                                     Senior Youth (Grade 10 and up)
                 Beatrice        Spun      Erin                                                 Early Youth (Grade 9 and below)
                 Ella            Lawrence  Goodlettsville                                        Senior Youth (Grade 10 and up)
                 Julianne        Erlewine  Gallatin First                                       Early Youth (Grade 9 and below)
                 Kaylee          Rumbley   Nashville Grace                                       Senior Youth (Grade 10 and up)
                 olivia          Nichols   Nashville Grace                                       Senior Youth (Grade 10 and up)

Age Level
Speech              First Name Last Name  What church are you a part of?                                      
Impromptu           Addie      Bennett    Nashville First (Main Campus)        Early Youth (Grade 9 and below)
                    Isabelle   Rector     Gallatin First                        Senior Youth (Grade 10 and up)
Oral Interpretation Addie      Bennett    Nashville First (Main Campus)        Early Youth (Grade 9 and below)
                    Addyson    Underwood  Concord Community                    Early Youth (Grade 9 and below)
                    Anna Grace Rector     Gallatin First                       Early Youth (Grade 9 and below)
                    Avery      Underwood  Concord Community                    Early Youth (Grade 9 and below)
                    Charley    Lawrence   Goodlettsville                       Early Youth (Grade 9 and below)
                    Emory      Duranceau  Clarksville First Community of Hope  Early Youth (Grade 9 and below)
                    Ian        Gibson     Columbia Grace                       Early Youth (Grade 9 and below)
                    Isabelle   Rector     Gallatin First                        Senior Youth (Grade 10 and up)
Original Oratory    Addie      Bennett    Nashville First (Main Campus)        Early Youth (Grade 9 and below)
                    Edwin      Reyes      Igelsia Del Nazareno Eben-Ezer       Early Youth (Grade 9 and below)
                    Isabelle   Rector     Gallatin First                        Senior Youth (Grade 10 and up)
Storytelling        Alice      Gooding    Clarksville Grace                    Early Youth (Grade 9 and below)
                    Isabelle   Rector     Gallatin First                        Senior Youth (Grade 10 and up)
                    Julianne   Erlewine   Gallatin First                       Early Youth (Grade 9 and below)
                    Keyira     Jackson    Hattiesburg First                    Early Youth (Grade 9 and below)
                    Lorelai    Cochran    Waverly                              Early Youth (Grade 9 and below)
                    Taylor     Banes      Hattiesburg First                    Early Youth (Grade 9 and below)
                    klarissa   mcfarland  Fly                                  Early Youth (Grade 9 and below)

Age Level
Academics                First Name          Last Name       What church are you a part of?                                                        
Math                     Adeline             Miller          Goodlettsville                                         Early Youth (Grade 9 and below)
                         Aidan               Barnes          Concord Community                                       Senior Youth (Grade 10 and up)
                         Alejandra Elizabeth Lopez Catalan   Glencliff Hispanic                                     Early Youth (Grade 9 and below)
                         Alice               Gooding         Clarksville Grace                                      Early Youth (Grade 9 and below)
                         Allister            Griffin         Spring Hill Restoration Community                       Senior Youth (Grade 10 and up)
                         Anna                Zirschky        Spring Hill Restoration Community                      Early Youth (Grade 9 and below)
                         Benjamin            Wentworth       Trevecca Community                                     Early Youth (Grade 9 and below)
                         Bennett             Hunter          Spring Hill Restoration Community                      Early Youth (Grade 9 and below)
                         Edwin               Reyes           Igelsia Del Nazareno Eben-Ezer       Early Youth (Grade 9 and below)Early Youth (Gr...
                         Eliseo              Herrera Xitumul Glencliff Hispanic                                      Senior Youth (Grade 10 and up)
                         Emily               Storie          Nashville First (Main Campus)                          Early Youth (Grade 9 and below)
                         Emory               Duranceau       Clarksville First Community of Hope                    Early Youth (Grade 9 and below)
                         Eva                 Webb            Gateway Community                                       Senior Youth (Grade 10 and up)
                         Gabriella           Phillips        Spring Hill Restoration Community                       Senior Youth (Grade 10 and up)
                         Hayden              Amos            Nashville First (Main Campus)                          Early Youth (Grade 9 and below)
                         Ian                 Gibson          Columbia Grace                                         Early Youth (Grade 9 and below)
                         Isabel              Lopez-Muñoz     Gallatin First                                          Senior Youth (Grade 10 and up)
                         Isabelle            Rector          Gallatin First                                          Senior Youth (Grade 10 and up)
                         Jaiden              Cochran         Waverly                                                 Senior Youth (Grade 10 and up)
                         Jason               Burch           Hermitage                                               Senior Youth (Grade 10 and up)
                         Jenna               Hartsock        Clarksville Grace                                      Early Youth (Grade 9 and below)
                         Jose                Camaja Perez    Glencliff Hispanic                                     Early Youth (Grade 9 and below)
                         Joshten             Pascacio        Igelsia Del Nazareno Eben-Ezer                          Senior Youth (Grade 10 and up)
                         Julianne            Erlewine        Gallatin First                                         Early Youth (Grade 9 and below)
                         Kathleen            Jeronimo        Gallatin First                                         Early Youth (Grade 9 and below)
                         Kinley              Hodge           Gallatin First                                         Early Youth (Grade 9 and b

...

Age Level
Vocal Music        What church are you a part of? First Name Last Name What church are you a part of?                                 
Singer/Song Writer Clarksville Grace              Alice      Gooding   Clarksville Grace               Early Youth (Grade 9 and below)
                                                  Allyson    Thomas    Clarksville Grace                Senior Youth (Grade 10 and up)
                                                  Bethany    Thomas    Clarksville Grace                Senior Youth (Grade 10 and up)
                   Concord Community              Kyla       Watson    Concord Community               Early Youth (Grade 9 and below)
                   Gallatin First                 Julianne   Erlewine  Gallatin First                  Early Youth (Grade 9 and below)
...                                                                                                                                ...
Worship Band       Memphis Grace                  Veronica   Ndikumana Memphis Grace                   Early Youth (Grade 9 and below)
                   Nashville First (Main Campus)  Chloe      Carawan   Nashville First (Main Campus)   Early Youth (Grade 9 and below)
                   Nashville Grace                David      Wood      Nashville Grace                  Senior Youth (Grade 10 and up)
                                                  Natalie    Reed      Nashville Grace                 Early Youth (Grade 9 and below)
                                                  Stephen    Purtee    Nashville Grace                  Senior Youth (Grade 10 and up)

[171 rows x 1 columns]

Age Level
Instrumental Music           What church are you a part of?      First Name Last Name    What church are you a part of?                                      
Instrumental Brass Solo      Crossroads                          Amie       Daniel       Crossroads                           Early Youth (Grade 9 and below)
                             Erin                                Jacob      Garza        Erin                                  Senior Youth (Grade 10 and up)
                                                                 kloe       shelton      Erin                                 Early Youth (Grade 9 and below)
                             Gateway Community                   Cooper     Landry       Gateway Community                    Early Youth (Grade 9 and below)
                                                                 Eva        Webb         Gateway Community                     Senior Youth (Grade 10 and up)
                             Spring Hill Restoration Community   Bennett    Hunter       Spring Hill Restoration Community    Early Youth (Grade 9 and below)
                             Trevecca Community                  Alex       Yusi         Trevecca Community                   Early Youth (Grade 9 and below)
Instrumental Group           Clarksville Grace                   Jenna      Hartsock     Clarksville Grace                    Early Youth (Grade 9 and below)
                             Columbia Grace                      Ian        Gibson       Columbia Grace                       Early Youth (Grade 9 and below)
Instrumental Percussion Solo Clarksville Grace                   Bethany    Thomas       Clarksville Grace                     Senior Youth (Grade 10 and up)
                             Columbia Grace                      Jackson    Johns        Columbia Grace                       Early Youth (Grade 9 and below)
                             Erin                                Benjamin   Meadow       Erin                                  Senior Youth (Grade 10 and up)
                             Gallatin First                      Wyatt      Johnson      Gallatin First                       Early Youth (Grade 9 and below)
                             Goodlettsville                      Matthew    Kropf        Goodlettsville                       Early Youth (Grade 9 and below)
                             Hattiesburg First                   David      Gavel        Hattiesburg First                    Early Youth (Grade 9 and below)
                             Nashville First (Main Campus)       Chloe      Carawan      Nashville First (Main Campus)        Early Youth (Grade 9 and below)
Instrumental Strings Solo    Erin                                Elijah     Spun         Erin                                 Early Youth (Grade 9 and below)
                                                                 Owen       Ziegler      Erin                                  Senior Youth (Grade 10 and up)
                             Gateway Community                   Elijah     McMillian    Gateway Community                     Senior Youth (Grade 10 and up)
                             Hendersonville                      Kenny      Gerdes       Hendersonville                        Senior Youth (Grade 10 and up)
                             Nashville Grace                     Keegan     Welch        Nashville Grace                       Senior Youth (Grade 10 and up)
                                                                 Keegan     Welch        Nashville Grace                       Senior Youth (Grade 10 and up)
                             Spring Hill Restoration Community   Allister   Griffin      Spring Hill Restoration Community     Senior Youth (Grade 10 and up)
Keyboard Solo                Clarksville Grace                   Allyson    Thomas       Clarksville Grace                     Senior Youth (Grade 10 and up)
                             Erin       

Empty DataFrame
Columns: []
Index: [(* Table Tennis, Junior High (ages 11-13), Adeline, Miller, Goodlettsville), (* Table Tennis, Junior High (ages 11-13), Antonio, Paulson, Goodlettsville), (* Table Tennis, Junior High (ages 11-13), Aubrey, Baggett, Columbia Grace), (* Table Tennis, Junior High (ages 11-13), Ben , Meikrantz, Hendersonville), (* Table Tennis, Junior High (ages 11-13), Eli, Ward, Memphis Grace), (* Table Tennis, Junior High (ages 11-13), Emry, Thompson, Gateway Community), (* Table Tennis, Junior High (ages 11-13), Jackson, Johns, Columbia Grace), (* Table Tennis, Junior High (ages 11-13), Jonah, Britton, Memphis Grace), (* Table Tennis, Junior High (ages 11-13), Lola Jane, Galloway, Nashville First (Main Campus)), (* Table Tennis, Middle High (ages 14-15), Avery, Underwood , Concord Community), (* Table Tennis, Middle High (ages 14-15), Claude, Galloway, Nashville First (Main Campus)), (* Table Tennis, Middle High (ages 14-15), Elijah, McMillian, Gateway Community), (* Table Tennis, Middle High (ages 14-15), Grey, Gernand, Gateway Community), (* Table Tennis, Middle High (ages 14-15), John, Wilkes, Nashville Grace), (* Table Tennis, Middle High (ages 14-15), Jonathan, Griffin, Nashville First (Main Campus)), (* Table Tennis, Middle High (ages 14-15), Landen, Boyd, Gateway Community), (* Table Tennis, Middle High (ages 14-15), Lily, Quick, Nashville First (Main Campus)), (* Table Tennis, Middle High (ages 14-15), Nataly, Rodriguez, Old Hickory), (* Table Tennis, Middle High (ages 14-15), William, Voss, Fly), (* Table Tennis, Senior High (ages 16-19), Addie , Peeples, Crossroads), (* Table Tennis, Senior High (ages 16-19), Allister , Griffin, Spring Hill Restoration Community), (* Table Tennis, Senior High (ages 16-19), Charles, Galloway, Nashville First (Main Campus)), (* Table Tennis, Senior High (ages 16-19), David, Wood, Nashville Grace), (* Table Tennis, Senior High (ages 16-19), Jason, Burch, Hermitage), (* Table Tennis, Senior High (ages 16-19), Kaden, Boyd, Gateway Community), (* Table Tennis, Senior High (ages 16-19), Kaylee, Rumbley, Nashville Grace), (* Table Tennis, Senior High (ages 16-19), Sam, Ruscio, Crossroads), (* Table Tennis, Senior High (ages 16-19), Warren, Bailey, Nashville First (Main Campus)), (* Table Tennis, Senior High (ages 16-19), Wes, Twining, Nashville First (Main Campus)), (* Tennis, Junior High (ages 11-13), Lucy , Howell, Nashville First (Main Campus)), (* Tennis, Middle High (ages 14-15), Avery, Underwood , Concord Community), (* Tennis, Middle High (ages 14-15), Dylan, Johnson, Nashville First (Main Campus)), (* Tennis, Senior High (ages 16-19), Cole, Petroff, Hermitage), (* Tennis, Senior High (ages 16-19), Giles, Knop, Hermitage), (* Tennis, Senior High (ages 16-19), Hudson, Stearns, Fly), (* Tennis, Senior High (ages 16-19), Jason, Burch, Hermitage), (* Tennis, Senior High (ages 16-19), faith , gill, Crossroads), (** Chess, Junior High (ages 11-13), Alice, Gooding, Clarksville Grace), (** Chess, Junior High (ages 11-13), Annalise, Thornsberry, Erin), (** Chess, Junior High (ages 11-13), Austin, Reigard, Hermitage), (** Chess, Junior High (ages 11-13), Eli, Ward, Memphis Grace), (** Chess, Junior High (ages 11-13), Hayden, Amos, Nashville First (Main Campus)), (** Chess, Junior High (ages 11-13), Jose , Camaja Perez, Glencliff Hispanic), (** Chess, Middle High (ages 14-15), Anna, Zirschky, Spring Hill Restoration Community), (** Chess, Middle High (ages 14-15), Hunter, Moore, Erin), (** Chess, Middle High (ages 14-15), Landen, Boyd, Gateway Community), (** Chess, Senior High (ages 16-19), Carson, Conwell, Erin), (** Chess, Senior High (ages 16-19), Eliseo, Herrera Xitumul, Glencliff Hispanic), (** Chess, Senior High (ages 16-19), Hunter , Jenkins , Trevecca Community), (** Chess, Senior High (ages 16-19), Isabelle, Rector, Gallatin First), (** Chess, Senior High (ages 16-19), Jason, Burch, Hermitage), (** Chess, Senior High (ages 16-19), Kaden, Boyd, Gateway Community), (**

Age Level Individual Sport
Team Sports        What church are you a part of?    First Name Last Name What church are you a part of?                                                     
* Co-Ed Dodgeball  Clarksville Grace                 Alice      Gooding   Clarksville Grace                                          Junior High (ages 11-13)
                                                     Jackson    Smith     Clarksville Grace                                          Middle High (ages 14-15)
                                                     Serenity   Halstead  Clarksville Grace                                          Senior High (ages 16-19)
                                                     alan       minor     Clarksville Grace                  Junior High (ages 11-13)Junior High (ages 11-13)
                   Columbia Grace                    Ian        Gibson    Columbia Grace                                             Middle High (ages 14-15)
...                                                                                                                                                       ...
** Co-Ed Gaga Ball Spring Hill Restoration Community Allister   Griffin   Spring Hill Restoration Community                          Senior High (ages 16-19)
                   Trevecca Community                Aiden      Pierc     Trevecca Community                                         Middle High (ages 14-15)
                                                     Cooper     Brown     Trevecca Community                                         Junior High (ages 11-13)
                                                     Rhylee     Colonello Trevecca Community                                         Junior High (ages 11-13)
                   Waverly                           Ainsley    Forester  Waverly                                                    Junior High (ages 11-13)

[538 rows x 1 columns]

Age Level
Quizzing              First Name Last Name   What church are you a part of?                                                        
* "A" League Quizzing Abbigale   Savage      Crossroads                                             Early Youth (Grade 9 and below)
                      Aundrea    Livingston  Erin                                                   Early Youth (Grade 9 and below)
                      Ben        Meikrantz   Hendersonville                                         Early Youth (Grade 9 and below)
                      Benjamin   Staples     Memphis Calvary                                        Early Youth (Grade 9 and below)
                      Carson     Wilkins     Waverly                                                Early Youth (Grade 9 and below)
                      Chandler   Pearce      Goodlettsville                                          Senior Youth (Grade 10 and up)
                      Chloe      Borja       Fly                                                    Early Youth (Grade 9 and below)
                      Connor     Thomas      Nashville First (Main Campus)                           Senior Youth (Grade 10 and up)
                      Edwin      Reyes       Igelsia Del Nazareno Eben-Ezer       Early Youth (Grade 9 and below)Early Youth (Gr...
                      Hudson     Stearns     Fly                                                     Senior Youth (Grade 10 and up)
                      Ian        Gibson      Columbia Grace                                         Early Youth (Grade 9 and below)
                      Isbell     Daniel      Hermitage                                               Senior Youth (Grade 10 and up)
                      Kaden      Boyd        Gateway Community                                       Senior Youth (Grade 10 and up)
                      Landen     Boyd        Gateway Community                                       Senior Youth (Grade 10 and up)
                      Stephen    Purtee      Nashville Grace                                         Senior Youth (Grade 10 and up)
                      klarissa   mcfarland   Fly                                                    Early Youth (Grade 9 and below)
* "B" League Quizzing Alice      Gooding     Clarksville Grace                                      Early Youth (Grade 9 and below)
                      Beatrice   Spun        Erin                                                   Early Youth (Grade 9 and below)
                      Emory      Duranceau   Clarksville First Community of Hope                    Early Youth (Grade 9 and below)
                      Max        Williams    Trevecca Community                                      Senior Youth (Grade 10 and up)
                      Nik        Ware        Hermitage                                               Senior Youth (Grade 10 and up)
                      Rudy       Colonello   Trevecca Community                                     Early Youth (Grade 9 and below)